<a href="https://colab.research.google.com/github/ladyTootie/ACE-R-D/blob/main/MPL_Final_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import TensorFlow and Check the Version

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


# Load and Preprocess NSL-KDD dataset

In [13]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential #linear stack of layers
from tensorflow.keras.layers import Dense #connected neural network layer

#Define column names
column_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
    'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
    'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds',
    'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
    'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'traffic_type', 'difficulty_level' # Adjusted column names
]

#Load the dataset
df = pd.read_csv('/content/KDDTrain+.txt', names=column_names)

# Drop the difficulty_level column
df = df.drop('difficulty_level', axis=1)

#Check if dataset loaded correctly
display(df.head())

#Identify categorical and numerical features
categorical_features = ['protocol_type', 'service', 'flag']
numerical_features = [col for col in df.columns if col not in categorical_features + ['traffic_type']]

# Convert numerical columns to numeric, coercing errors, and fill NaNs
for col in numerical_features:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(df[col].mean()) # or .median()

# Separate features (X) and labels (y)
X = df.drop('traffic_type', axis=1)
y = df['traffic_type']

# Preprocessing: One-hot encode categorical features and scale numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

# Apply preprocessing
X_processed = preprocessor.fit_transform(X)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_encoded, test_size=0.2, random_state=42)

#print("Preprocessing complete. Data split into training and testing sets.")
#print("Shape of X_train:", X_train.shape)
#print("Shape of y_train:", y_train.shape)
#print("Shape of X_test:", X_test.shape)
#print("Shape of y_test:", y_test.shape)



,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,traffic_type
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


Preprocessing complete. Data split into training and testing sets.
Shape of X_train: (100778, 122)
Shape of y_train: (100778,)
Shape of X_test: (25195, 122)
Shape of y_test: (25195,)


# Create the MLP Model


In [14]:
# Get the number of input features
input_shape = X_train.shape[1]

# Get the number of output classes
output_shape = len(label_encoder.classes_)

# Create the MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape,)), # Input layer and first hidden layer
    Dense(64, activation='relu'), # Second hidden layer
    Dense(output_shape, activation='softmax') # Output layer with softmax for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Use sparse_categorical_crossentropy for integer labels
              metrics=['accuracy'])

# Print the model summary
#model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        15,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 23)             │         1,495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,495 (99.59 KB)

 Trainable params: 25,495 (99.59 KB)

 Non-trainable params: 0 (0.00 B)

# Train the Model

In [15]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

print("Model training complete.")

Epoch 1/10
2520/2520 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9517 - loss: 0.2380 - val_accuracy: 0.9908 - val_loss: 0.0346
Epoch 2/10
2520/2520 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9908 - loss: 0.0323 - val_accuracy: 0.9922 - val_loss: 0.0289
Epoch 3/10
2520/2520 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9933 - loss: 0.0211 - val_accuracy: 0.9936 - val_loss: 0.0229
Epoch 4/10
2520/2520 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9932 - loss: 0.0200 - val_accuracy: 0.9944 - val_loss: 0.0225
Epoch 5/10
2520/2520 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9946 - loss: 0.0177 - val_accuracy: 0.9945 - val_loss: 0.0232
Epoch 6/10
2520/2520 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9947 - loss: 0.0162 - val_accuracy: 0.9906 - val_loss: 0.0333
Epoch 7/10
2520/2520 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9952 - loss: 0.0153 - val_accuracy: 0.9942 - val_loss: 0.0248
Epoch 8/10
2520/2520 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9954 - loss: 0.0150 -

In [16]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

print("Model evaluation complete.")
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Model evaluation complete.
Test Loss: 0.0262
Test Accuracy: 0.9934


# Preprocess the conn.log

In [ ]:
# Assuming your conn.log file is named 'conn.log' and is in the /content directory
# and has the same column structure as the training data (including the extra column if present)

# Define the same column names including the placeholder for the extra column if it exists
# Using the column_names defined earlier in the notebook
# column_names = [ ... ]

# Load the new conn.log data
conn_df = pd.read_csv('/content/conn.log', names=column_names)

# Drop the extra column if it was included in column_names
if 'extra_column' in conn_df.columns:
    conn_df = conn_df.drop('extra_column', axis=1)

# Drop the difficulty_level column if it exists and is not needed for prediction
if 'difficulty_level' in conn_df.columns:
    conn_df = conn_df.drop('difficulty_level', axis=1)

# Check if the loaded data has the expected columns
# This is a good practice to ensure consistency with training data
expected_features = [col for col in column_names if col not in ['extra_column', 'traffic_type', 'difficulty_level']]
if not all(col in conn_df.columns for col in expected_features):
    print("Warning: Columns in conn.log do not match expected features.")
    # You might want to handle this mismatch, e.g., by re-aligning columns or raising an error

# Identify numerical and categorical features (re-using the lists defined earlier)
# categorical_features = ['protocol_type', 'service', 'flag']
# numerical_features = [col for col in conn_df.columns if col not in categorical_features]

# Convert numerical columns to numeric, coercing errors, and fill NaNs
# It's important to use the mean/median from the TRAINING data here
# For simplicity in this example, we'll re-calculate, but for production, use training stats
for col in numerical_features:
    conn_df[col] = pd.to_numeric(conn_df[col], errors='coerce')
    conn_df[col] = conn_df[col].fillna(df[col].mean()) # Use mean from original training df

# Apply the *fitted* preprocessor to the new data
# Make sure X contains only the features needed for the model
X_conn = conn_df[numerical_features + categorical_features]

# Apply the preprocessing transformation
X_conn_processed = preprocessor.transform(X_conn)

print("Preprocessing of conn.log complete.")
print("Shape of preprocessed conn.log data:", X_conn_processed.shape)

# The preprocessed data is now in X_conn_processed, ready for prediction